In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!rm -rf clothing-dataset-small/
!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

In [ ]:
img = load_img('./clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg', target_size=(299, 299))
img

In [ ]:
x = np.array(img)
x.shape

In [ ]:
model = Xception(
    weights='imagenet',
    input_shape=(299, 299, 3)
)

In [ ]:
X = np.array([x])
X.shape

In [ ]:
X = preprocess_input(X)
X.shape

In [ ]:
pred = model.predict(X)
pred.shape

In [ ]:
decode_predictions(pred)

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train/', target_size=(150, 150), batch_size=32
)

In [ ]:
train_ds.class_indices

In [ ]:
X, y = next(train_ds)

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation/', target_size=(150, 150), batch_size=32, shuffle=False
)

In [ ]:
base_model = Xception(
    weights='imagenet', include_top=False, input_shape=(150, 150, 3)
)
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(10)(vectors)
model = keras.Model(inputs, outputs)

In [ ]:
preds = model.predict(X)

In [ ]:
preds.shape

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
history.history

In [ ]:
# plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(
    weights='imagenet', include_top=False, input_shape=(150, 150, 3)
    )
    base_model.trainable = False
    ###########################################
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    outputs = keras.layers.Dense(10)(inner)
    ###########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])
    return model

In [ ]:
scores = {}
for lr in [0.0001, 0.001, 0.01, 0.1]:
    print(f'Learning rate: {lr}')
    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    print()

In [ ]:
del scores[0.1]
del scores[0.0001]

In [ ]:
scores

In [ ]:
for lr, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=lr)
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001

In [ ]:
model.save_weights('model_v1.weights.h5')

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001
model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[checkpoint])

In [ ]:
learning_rate = 0.001
scores = {}
for size in [10, 100, 1000]:
    print(f'Inner layer size: {size}')
    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=f'val={size}')
plt.xticks(np.arange(10))
plt.yticks([0.78, 0.80, 0.82, 0.825, 0.83])
plt.legend()

In [ ]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
    weights='imagenet', include_top=False, input_shape=(150, 150, 3)
    )
    base_model.trainable = False
    ###########################################
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    outputs = keras.layers.Dense(10)(drop)
    ###########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])
    return model

In [ ]:
learning_rate = 0.001
size = 100

scores = {}
for droprate in [0.0, 0.2, 0.5, 0.8]:
    print(f'Droprate: {droprate}')
    model = make_model(learning_rate=learning_rate, size_inner=size, droprate=droprate)
    history = model.fit(train_ds, epochs=30, validation_data=val_ds)
    scores[droprate] = history.history
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=f'val={droprate}')
# plt.xticks(np.arange(10))
# plt.yticks([0.78, 0.80, 0.82, 0.825, 0.83])
plt.ylim(0.78, 0.86)
plt.legend()

In [ ]:
learning_rate = 0.001
size_inner = 100
droprate = 0.2

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    # rotation_range=30,
    # width_shift_range=10.0,
    # height_shift_range=10.0,
    shear_range=10,
    zoom_range=0.1,
    vertical_flip=True
)
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train/', target_size=(150, 150), batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation/', target_size=(150, 150), batch_size=32, shuffle=False
)

In [ ]:
learning_rate = 0.001
size_inner = 100
droprate = 0.2
model = make_model(learning_rate=learning_rate, size_inner=size_inner, droprate=droprate)
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
hist = history.history
plt.plot(hist['val_accuracy'], label='val')
plt.plot(hist['accuracy'], label='train')
plt.legend()

In [ ]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
    weights='imagenet', include_top=False, input_shape=(input_size, input_size, 3)
    )
    base_model.trainable = False
    ###########################################
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    outputs = keras.layers.Dense(10)(drop)
    ###########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])
    return model

In [ ]:
input_size = 299

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    vertical_flip=True
)
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train/', target_size=(input_size, input_size), batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation/', target_size=(input_size, input_size), batch_size=32, shuffle=False
)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_{epoch:02d}_{val_accuracy:.3f}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001
size_inner = 100
droprate = 0.2

model = make_model(input_size=input_size, learning_rate=learning_rate, size_inner=size_inner, droprate=droprate)
history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[checkpoint])

In [ ]:
trained_model = keras.models.load_model('xception_v4_36_0.883.keras')
input_size = 299
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_ds = test_gen.flow_from_directory(
    './clothing-dataset-small/test/', target_size=(input_size, input_size), batch_size=32, shuffle=False
)
trained_model.evaluate(test_ds)

In [ ]:
path = './clothing-dataset-small/test/pants/c8d21106-bbdb-4e8d-83e4-bf3d14e54c16.jpg'
img = load_img(path, target_size=(input_size, input_size))
x = np.array(img)
X = np.array([x])
# print(X.shape)
X = preprocess_input(X)
pred = trained_model.predict(X)

In [ ]:
classes = ['dress','hat','longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

In [ ]:
dict(zip(classes, pred[0]))